In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import r2_score
from skorch import NeuralNetRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
# load training data
Xtr_loadpath = 'Xtr.csv'
Xts_loadpath = 'Xts.csv'
ytr_loadpath = 'ytr.csv'

Xtr = np.loadtxt(Xtr_loadpath, delimiter=",")
Xts = np.loadtxt(Xts_loadpath, delimiter=",")
ytr = np.loadtxt(ytr_loadpath, delimiter=",")

In [3]:
# standardize the training data
Xtr_mean = np.mean(Xtr,axis=0)
Xtr_std = np.std(Xtr,axis=0)
Xtr_standardized = ((Xtr-Xtr_mean[None,:])/Xtr_std[None,:]) # revise this line as needed
Xts_standardized = ((Xts-Xtr_mean[None,:])/Xtr_std[None,:]) # revise this line as needed

# save the standardized training data
Xtr_savepath = 'Xtr_pytorch.csv'
Xts_savepath = 'Xts_pytorch.csv'
ytr_savepath = 'ytr_pytorch.csv'
yts_hat_savepath = 'yts_hat_pytorch.csv'

np.savetxt(Xtr_savepath, Xtr_standardized, delimiter=",")
np.savetxt(Xts_savepath, Xts_standardized, delimiter=",")
np.savetxt(ytr_savepath, ytr, delimiter=",")

In [4]:
## Hyper Parameter Gird Search Cross-Validation

import sklearn.model_selection 

# construct repeated k-fold cross-val object
nfold = 10
nrep = 5
rkf = sklearn.model_selection.RepeatedKFold(n_splits=nfold,
                                            n_repeats=nrep, random_state=1)

# Define your PyTorch neural network as a class
class Net(nn.Module):
    def __init__(self,nin,nh,nout):
        super(Net,self).__init__()
        self.sigmoid = nn.Sigmoid()
        self.Dense1 = nn.Linear(nin,nh)
        self.Dense2 = nn.Linear(nh,nout)
        
    def forward(self,x):
        x = self.sigmoid(self.Dense1(x))
        out = self.Dense2(x)
        return out

# Create a dictionary of hyperparameters and their possible values
##nh1_space = np.linspace(20,60,1)
##nh2_space = np.linspace(10,40,1)
##nh3_space = np.linspace(5,20,1)
#criterion_space = [torch.nn.MSELoss, torch.nn.L1Loss, torch.nn.HuberLoss]
criterion_space = [torch.nn.BCELoss, torch.nn.CrossEntropyLoss]

param_grid = {
    
    'criterion': criterion_space
    #'optimizer': [optim.Adam, optim.SGD]
    #'optimizer_lr': [0.01, 0.001],
}

# Create a PyTorch neural network object using the above-defined class
cv_net = NeuralNetRegressor(
    Net,
    max_epochs=25,
    #criterion=nn.MSELoss,
)

# Create a GridSearchCV object using the neural network and the hyperparameter dictionary
gscv = GridSearchCV(cv_net, param_grid, cv=rkf, scoring='r2')

# Fit the GridSearchCV object to your data
gscv.fit(Xtr_standardized, ytr)

# Print the best hyperparameters and the corresponding R-squared score
print(gs.best_params_, gs.best_score_)

C:\Users\BassD\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\BassD\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\BassD\anaconda3\lib\site-packages\skorch\regressor.py", line 91, in fit
    return super(NeuralNetRegressor, self).fit(X, y, **fit_params)
  File "C:\Users\BassD\anaconda3\lib\site-packages\skorch\net.py", line 1228, in fit
    self.initialize()
  File "C:\Users\BassD\anaconda3\lib\site-pa

TypeError: __init__() missing 3 required positional arguments: 'nin', 'nh', and 'nout'

In [9]:
# Define your PyTorch neural network as a class
class MyNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Create a dictionary of hyperparameters and their possible values
params = {
    'module__hidden_dim': [16, 32, 64],
    'optimizer': [optim.Adam, optim.SGD],
    'optimizer__lr': [0.01, 0.001],
}

# Create a PyTorch neural network object using the above-defined class
net = NeuralNetRegressor(
    MyNet,
    criterion=nn.MSELoss,
)

# Create a GridSearchCV object using the neural network and the hyperparameter dictionary
gs = GridSearchCV(net, params, cv=3, scoring='r2')

# Fit the GridSearchCV object to your data
gscv.fit(Xtr_standardized, ytr)

# Print the best hyperparameters and the corresponding R-squared score
print(gs.best_params_, gs.best_score_)

C:\Users\BassD\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\BassD\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\BassD\anaconda3\lib\site-packages\skorch\regressor.py", line 91, in fit
    return super(NeuralNetRegressor, self).fit(X, y, **fit_params)
  File "C:\Users\BassD\anaconda3\lib\site-packages\skorch\net.py", line 1228, in fit
    self.initialize()
  File "C:\Users\BassD\anaconda3\lib\site-pa

TypeError: __init__() missing 3 required positional arguments: 'nin', 'nh', and 'nout'